In [1]:
 # Import libraries
import pandas as pd
import pickle as pk

# import libraries for api
import os
from dotenv import load_dotenv 
import requests
import json

# import libraries for vizualization
from pprint import pprint

In [2]:
# Import API

In [3]:
## Get API Keys
load_dotenv("./environment/.env")

rapid_api_key = os.getenv("rapid_key")
rapid_api_host = os.getenv("rapid_host")

print(rapid_api_key)

cb07c5da44msh5da7167914d0ae7p1acdc5jsna1515709e811


In [4]:
## Zillow currunt ---------------------------------------------------------------------------------------------------

In [5]:
### Local & constants for zillow API
home_type = "Houses"
city = "philadelphia"
state = "pa"
search_str = city + "," + state

print( f"search string : {home_type} in {search_str}")

search string : Houses in philadelphia,pa


In [6]:
### Get Data from the API found on rapidAPI with API function ExtendedSearch (This is un undirect acces to Zillow)

url_extended_search = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring_extended_search = {"location":search_str ,
                               "home_type":home_type}

headers = {
    'x-rapidapi-host': rapid_api_host,
    'x-rapidapi-key': rapid_api_key
    }

response_extended_search = requests.request("GET", url_extended_search, headers=headers, params=querystring_extended_search)

In [8]:
### Store data in a pickle: Zillow_data
filename = "Zillow_data"
outfile = open(filename, "wb")

pk.dump(response_extended_search,outfile)
outfile.close()

In [10]:
### Open Pickle to make a list ZPID for the houses currently for sale
# Open Pickle
filename = "Zillow_data"
infile = open(filename,'rb')
new_dict = pk.load(infile)
infile.close()

#Transform data to Json
z_for_sale_resp =new_dict.json()

# Transform Json data to DataFrame
current_zillow_df = pd.json_normalize(data=z_for_sale_resp["props"])
print('num of rows:', len(current_zillow_df))
print('num of columns:', len(current_zillow_df.columns))

# make a list of the zpid as key to extract the data from the API similar_search, every ZPID is unique
key = current_zillow_df["zpid"].tolist()
print(key)

num of rows: 40
num of columns: 21
['10360239', '2068504637', '10584074', '10359239', '10186985', '122287109', '2067555864', '10519350', '10285435', '10543196', '2067980164', '10573910', '10438532', '2071261604', '10363453', '10515845', '118366442', '10533805', '10466123', '118371376', '10534810', '10232128', '10526147', '2067653448', '10291765', '80946305', '10231431', '10486635', '10529618', '2111077001', '165109684', '10285174', '10526141', '10267017', '10486771', '10294716', '10279739', '118352000', '10290693', '10511158']


In [11]:
## Zillow similar search : similar recently sold ----------------------------------------------------------------------------------------------------

In [12]:
### Get Data from the API found on rapidAPI with API function SimilarSales (This is un undirect acces to Zillow)

url_similar_house = "https://zillow-com1.p.rapidapi.com/similarSales"

#Initiate a new dictionnary to contain the data that is goind to be extracted
zpid_dict = {}

for zpid in key:
    # Create a query with one of the ZPID of the list key
    querystring_similar_house = {"zpid":zpid}
    
    # Put the query in the API and save the data in the variable response_similar_house
    response_similar_house = requests.request("GET", url_similar_house, headers=headers, params = querystring_similar_house)
    
    # Add the extracted data in a dictionnary that contains all the calls to the API
    zpid_dict[zpid] = response_similar_house.json()

In [13]:
### Store data in a pickle : Zillow_old_data
filename_old = "Zillow_sold_data"
outfile = open(filename_old, "wb")

pk.dump(zpid_dict,outfile)
outfile.close()

In [14]:
print(zpid_dict)

{'10360239': [{'dateSold': 1626825600000, 'bedrooms': 4, 'homeStatus': 'PENDING', 'latitude': 40.052106, 'miniCardPhotos': [{'url': 'https://photos.zillowstatic.com/fp/e1e4251d4f7031f5a2b7ef8421393f21-p_c.jpg'}], 'zpid': 10360189, 'longitude': -75.101203, 'bathrooms': 4, 'address': {'city': 'Philadelphia', 'state': 'PA', 'streetAddress': '6328 Newtown Ave', 'zipcode': '19111'}, 'livingArea': 2216, 'livingAreaUnits': 'Square Feet', 'homeType': 'SINGLE_FAMILY', 'currency': 'USD', 'lastSoldPrice': 325000, 'price': 399999}, {'dateSold': 1607990400000, 'bedrooms': 5, 'homeStatus': 'RECENTLY_SOLD', 'latitude': 40.052383, 'miniCardPhotos': [{'url': 'https://photos.zillowstatic.com/fp/cf6f5017b014deda2562f3c08bf950c9-p_c.jpg'}], 'zpid': 2077604490, 'longitude': -75.098753, 'bathrooms': 3, 'address': {'city': 'Philadelphia', 'state': 'PA', 'streetAddress': '6430 Hasbrook Ave #A', 'zipcode': '19111'}, 'livingArea': 1950, 'livingAreaUnits': 'Square Feet', 'homeType': 'SINGLE_FAMILY', 'currency': 

In [14]:


# Extracting data in working file must be adapted with a for loop.
combined_similar= pd.DataFrame()

for key in tot_zpid:
    
    df_similar= pd.json_normalize(data= tot_zpid[key])
    df_similar["origine ZPID"] = key
    combined_similar = pd.concat( [combined_similar, df_similar], axis=0, join = "outer")
    

combined_similar.reset_index(inplace=True) 

print('num of rows:', len(combined_similar))
print('num of columns:', len(combined_similar.columns))
combined_similar.head(10)


num of rows: 200
num of columns: 20


,index,dateSold,bedrooms,homeStatus,latitude,miniCardPhotos,zpid,longitude,bathrooms,livingArea,livingAreaUnits,homeType,currency,lastSoldPrice,price,address.city,address.state,address.streetAddress,address.zipcode,origine ZPID
0,0,1610582400000,5,RECENTLY_SOLD,40.055460,[{'url': 'https://photos.zillowstatic.com/fp/1...,10214276,-75.195090,3,2138,Square Feet,SINGLE_FAMILY,USD,470000,470000,Philadelphia,PA,7139 Lincoln Dr,19119,10428068
1,1,1619740800000,3,RECENTLY_SOLD,40.051693,[{'url': 'https://photos.zillowstatic.com/fp/f...,10284714,-75.194620,4,2200,Square Feet,SINGLE_FAMILY,USD,477500,477500,Philadelphia,PA,403 W Ellet St,19119,10428068
2,2,1614556800000,4,RECENTLY_SOLD,40.051390,[{'url': 'https://photos.zillowstatic.com/fp/6...,10284790,-75.195480,3,2400,Square Feet,SINGLE_FAMILY,USD,443000,443000,Philadelphia,PA,426 Glen Echo Rd,19119,10428068
3,3,1629158400000,5,RECENTLY_SOLD,40.057297,[{'url': 'https://photos.zillowstatic.com/fp/c...,103779525,-75.193504,3,2665,Square Feet,SINGLE_FAMILY,USD,480000,480000,Philadelphia,PA,7141 Cresheim Rd,19119,10428068
4,4,1634256000000,3,RECENTLY_SOLD,40.051323,[{'url': 'https://photos.zillowstatic.com/fp/4...,333709815,-75.196860,3,1760,Square Feet,SINGLE_FAMILY,USD,460000,460000,Philadelphia,PA,504 W Mount Pleasant Ave,19119,10428068
5,0,1610582400000,5,RECENTLY_SOLD,40.055460,[{'url': 'https://photos.zillowstatic.com/fp/1...,10214276,-75.195090,3,2138,Square Feet,SINGLE_FAMILY,USD,470000,470000,Philadelphia,PA,7139 Lincoln Dr,19119,10291269
6,1,1619740800000,3,RECENTLY_SOLD,40.051693,[{'url': 'https://photos.zillowstatic.com/fp/f...,10284714,-75.194620,4,2200,Square Feet,SINGLE_FAMILY,USD,477500,477500,Philadelphia,PA,403 W Ellet St,19119,10291269
7,2,1614556800000,4,RECENTLY_SOLD,40.051390,[{'url': 'https://photos.zillowstatic.com/fp/6...,10284790,-75.195480,3,2400,Square Feet,SINGLE_FAMILY,USD,443000,443000,Philadelphia,PA,426 Glen Echo Rd,19119,10291269
8,3,1629158400000,5,RECENTLY_SOLD,40.057297,[{'url': 'https://photos.zillowstatic.com/fp/c...,103779525,-75.193504,3,2665,Square Feet,SINGLE_FAMILY,USD,480000,480000,Philadelphia,PA,7141 Cresheim Rd,19119,10291269
9,4,1634256000000,3,RECENTLY_SOLD,40.051323,[{'url': 'https://photos.zillowstatic.com/fp/4...,333709815,-75.196860,3,1760,Square Feet,SINGLE_FAMILY,USD,460000,460000,Philadelphia,PA,504 W Mount Pleasant Ave,19119,10291269
